# Recherche des parcours types des utilisateurs sur le cdtn

**Définition** : On souhaite définir quels sont les parcours les plus populaires sur le site en utilisant les données de matomo. On peut découper le parcours en plusieurs étapes : Un point d'entrée puis une sucession de navigation / recherches dans les pages du cdtn et visites sur les pages correspondantes.

Les utilisateurs peuvent entrer sur le CDTN par :
- Moteurs de recherche
- Une entrée directe
- un site web
- les réseaux sociaux
- une campagne
- je ne sais pas si on a un moyen de voir ceux qui arriveraient via le widget CDTN intégré sur un site tier

Une fois dans le cdtn, les utilisateurs peuvent naviguer en interne par :
- une recherche sur le cdtn
- navigation via les thèmes

Les types de pages que peuvent consulter les utilisateurs au cours du parcours :
- la home
- les outils
- les contributions
- les modèles de courriers
- les convention collective
- les fiches service-public
- les fiches ministère travail
- les pages information
- les dossiers
- le glossaire
- les articles "code-du-travail"


## On commence par le chargement des données
on décommente ce dont on a besoin

In [ ]:
import pandas as pd
import numpy as np
import time
import json
import math
from tqdm.notebook import tqdm
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import itertools
import time
tqdm.pandas()

#from typing import List, Dict


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', 300)

## On récupère les données via celle déjà télécharger

In [ ]:
from join_multiple_file import pd_read_pattern, PATH_CDTN_MONOLOG

logs_january = pd_read_pattern(PATH_CDTN_MONOLOG + "/data-all-logs-january/*")
logs_february = pd_read_pattern(PATH_CDTN_MONOLOG + "/data-all-logs-february/*")
logs_march = pd_read_pattern(PATH_CDTN_MONOLOG + "/data-all-logs-march/*")
logs_april = pd_read_pattern(PATH_CDTN_MONOLOG + "/data-all-logs-april/*")
logs_may = pd_read_pattern(PATH_CDTN_MONOLOG + "/data-all-logs-may/*")
logs_june = pd_read_pattern(PATH_CDTN_MONOLOG + "/data-all-logs-june/*")
logs_july = pd_read_pattern(PATH_CDTN_MONOLOG + "/data-all-logs-july/*")

In [ ]:
def format_url_and_get_month_and_datetime_columns(df: pd.DataFrame):
    df['url'] = df['url'].str.split('#').str[0].str.split('?').str[0]
    df['month'] = pd.DatetimeIndex(df['logfile']).month
    df['datetime'] = df['timestamp'].apply(datetime.fromtimestamp)
    return df

In [ ]:
def clean_dataset_from_useless_rows(df):
    df_shortened = df[df['type'].isin(['visit_content', 'outil', 'themes', 'select_result', 'search', 'home',
                                    'pagecc_clickcontrib', 'modeles_courriers'])]
    return df_shortened.loc[
        (df_shortened['idVisit'].shift() != df_shortened['idVisit']) |
        (df_shortened['url'].shift() != df_shortened['url']) |
        (df_shortened['type'].shift() != df_shortened['type']) |
        (df_shortened['referrerTypeName'].shift() != df_shortened['referrerTypeName'])
    ]

In [ ]:
logs = pd.concat([logs_january, logs_february, logs_march, logs_april, logs_may, logs_june, logs_july]).copy()

In [ ]:
logs = format_url_and_get_month_and_datetime_columns(logs)

In [ ]:
cleaned_logs = clean_dataset_from_useless_rows(logs.copy())

In [ ]:
#logs_without_duplicates = logs.drop_duplicates(subset=['idVisit', 'url', 'type']).copy()
#logs_without_duplicates.shape[0]

# Exploration des données 

In [ ]:
logs.shape

In [ ]:
logs.head()

In [ ]:
cleaned_logs.shape

In [ ]:
cleaned_logs.head()

Afin de restreindre le champs des possibles et commencer par petits incréments, on se propose d'analyser la première quinzaine de juillet

In [ ]:
cleaned_logs_july = cleaned_logs[
    (cleaned_logs['logfile']>'2022-06-30') &
    (cleaned_logs['logfile']<'2022-08-01')
].copy()

In [ ]:
logs_first_fortnight_july = cleaned_logs[
    (cleaned_logs['logfile']>'2022-06-30') &
    (cleaned_logs['logfile']<'2022-07-16')
].copy()

## Commencons par travailler sur le point d'entrée

On souhaite obtenir un point d'entrée sous la forme d'un élément de la liste suivante : 
- Moteurs de recherche
- Une entrée directe
- un site web
- les réseaux sociaux
- une campagne

In [ ]:
def get_entry_point(df: pd.DataFrame):
    return df.referrerTypeName.iloc[0]

In [ ]:
logs_first_fortnight_july[:60].groupby('idVisit').apply(get_entry_point)

## Continuons sur la suite du parcours utilisateur (actions qu'il peut faire ou type de pages qu'il peut voir)

On souhaite récupérer une liste d'éléments représentant les étapes sur lesquels passent l'utilisateur :

Les étapes peuvent être le fait de vouloir de naviguer entre les pages en utilisant la recherche ou les thèmes :
- [ ] une recherche sur le cdtn
- [ ] navigation via les thèmes

Ou bien le contenu que l'utilisateur visite :
- [X] la home
- [X] les outils
- [X] les contributions
- [X] les modèles de courriers
- [X] les convention collective
- [X] les fiches service-public
- [X] les fiches ministère travail
- [X] les pages information
- [X] les dossiers
- [X] le glossaire
- [X] les articles "code-du-travail"
- [X] les articles "droits-du-travail"

In [ ]:
sep_char = '_'
urls = {
    'outils': 'https://code.travail.gouv.fr/outils',
    'contribution_with_/': 'https://code.travail.gouv.fr/contribution/',
    'modeles_de_courriers': 'https://code.travail.gouv.fr/modeles-de-courriers',
    'convention_collective_with_/': 'https://code.travail.gouv.fr/convention-collective/',
    'fiche_service_public_with_/': 'https://code.travail.gouv.fr/fiche-service-public/',
    'fiche_ministere_travail_with_/': 'https://code.travail.gouv.fr/fiche-ministere-travail/',
    'home': 'https://code.travail.gouv.fr/',
    'code_du_travail_with_/': 'https://code.travail.gouv.fr/code-du-travail/',
    'information_with_/': 'https://code.travail.gouv.fr/information/',
    'glossaire': 'https://code.travail.gouv.fr/glossaire',
    'dossiers_with_/': 'https://code.travail.gouv.fr/dossiers/',
    'droit_du_travail': 'https://code.travail.gouv.fr/droit-du-travail',
    'themes': 'https://code.travail.gouv.fr/themes'
}

urls_len = {
    'outils': len(urls['outils']),
    'contribution_with_/': len(urls['contribution_with_/']),
    'modeles_de_courriers': len(urls['modeles_de_courriers']),
    'convention_collective_with_/': len(urls['convention_collective_with_/']),
    'fiche_service_public_with_/': len(urls['fiche_service_public_with_/']),
    'fiche_ministere_travail_with_/': len(urls['fiche_ministere_travail_with_/']),
    'home': len(urls['home']),
    'code_du_travail_with_/': len(urls['code_du_travail_with_/']),
    'information_with_/': len(urls['information_with_/']),
    'glossaire': len(urls['glossaire']),
    'dossiers_with_/': len(urls['dossiers_with_/']),
    'droit_du_travail': len(urls['droit_du_travail']),
    'themes': len(urls['themes']),
}

def get_page_from_url(url, pattern_to_remove):
    return url.split(pattern_to_remove)[1].split('/')[0]

def is_my_url_domain_and_subdirectory_finishing_by_a_slash(url, url_key_domain):
    return urls_len[url_key_domain] < len(url) and url[urls_len[url_key_domain]] == '/'
    

def get_action_from_series(log: pd.Series): 
    url = log.url
    if(log.type == 'visit_content'):
        if url.startswith(urls['outils']):
            if url == urls['outils']:
                return 'Boite à outils'
            elif is_my_url_domain_and_subdirectory_finishing_by_a_slash(url, 'outils'):
                return "Visite d'un outil" + sep_char + get_page_from_url(url, urls['outils'] + '/')
        
        elif url.startswith(urls['contribution_with_/']):
            return "Visite d'une contribution" + sep_char + get_page_from_url(url, urls['contribution_with_/'])
        
        elif url.startswith(urls['modeles_de_courriers']):
            if url == urls['modeles_de_courriers']:
                return 'Boite de modèles de courriers'
            elif is_my_url_domain_and_subdirectory_finishing_by_a_slash(url, 'modeles_de_courriers'):
                return 'Visite de modèle de courrier' + sep_char + get_page_from_url(url, urls['modeles_de_courriers'] + '/')
        
        elif url.startswith(urls['convention_collective_with_/']):
            return 'Visite de convention collective' + sep_char + get_page_from_url(url, urls['convention_collective_with_/'])
        
        elif url.startswith(urls['fiche_service_public_with_/']):
            return 'Visite de fiche service public' + sep_char + get_page_from_url(url, urls['fiche_service_public_with_/'])
        
        elif url.startswith(urls['fiche_ministere_travail_with_/']):
            return 'Visite de fiche ministère travail' + sep_char + get_page_from_url(url, urls['fiche_ministere_travail_with_/'])
        
        elif url == urls['home']:
            return 'Visite de la home'
        
        elif url.startswith(urls['code_du_travail_with_/']):
            return "Visite d'un article code-du-travail" + sep_char + get_page_from_url(url, urls['code_du_travail_with_/'])
        
        elif url.startswith(urls['information_with_/']):
            return 'Visite de page information' + sep_char + get_page_from_url(url, urls['information_with_/'])
        
        elif url.startswith(urls['glossaire']):
            return 'Visite du glossaire' 
        
        elif url.startswith(urls['dossiers_with_/']):
            return "Visite d'un dossier" + sep_char + get_page_from_url(url, urls['dossiers_with_/'])
        
        elif url.startswith(urls['droit_du_travail']):
            return 'Visite de la page droit du travail'
        
    elif(log.type == 'outil'):  
        if url.startswith(urls['outils']):
            if url == urls['outils']:
                return 'Boite à outils'
            elif is_my_url_domain_and_subdirectory_finishing_by_a_slash(url, 'outils'):
                return "Visite d'un outil" + sep_char + get_page_from_url(url, urls['outils'] + '/')
    
    elif(log.type == 'themes'): 
        if url.startswith(urls['themes']):
            if url == urls['themes']:
                return 'Boite à themes'
            elif is_my_url_domain_and_subdirectory_finishing_by_a_slash(url, 'themes'):
                return "Visite d'un thème" + sep_char + get_page_from_url(url, urls['themes'] + '/')
        
    elif log.type == 'select_result' or log.type == 'search': 
        return 'Usage du moteur de recherche'
    
    elif log.type == 'home':  
        return 'Visite de la home'
    
    elif log.type == 'pagecc_clickcontrib': 
        if url.startswith(urls['convention_collective_with_/']):
            return 'Visite de convention collective' + sep_char + get_page_from_url(url, urls['convention_collective_with_/'])
            
    elif log.type == 'modeles_courriers':  
        if url == urls['modeles_de_courriers']:
            return 'Boite de modèles de courriers'
        elif is_my_url_domain_and_subdirectory_finishing_by_a_slash(url, 'modeles_de_courriers'):
            return 'Visite de modèle de courrier' + sep_char + get_page_from_url(url, urls['modeles_de_courriers'] + '/')
    
    return np.NaN
    
def get_list_of_actions(df: pd.DataFrame):
    actions_list = df.apply(get_action_from_series, axis=1).dropna().tolist()
    return [key.split(sep_char)[0] for key,_value in itertools.groupby(actions_list)]


In [ ]:
logs_first_fortnight_july[:100].groupby('idVisit').apply(get_list_of_actions)

## Synthèse du parcours utilisateur

In [ ]:
def get_user_journey(df: pd.DataFrame):
    s = pd.DataFrame()
    s['entry_point'] = [get_entry_point(df)]
    s['actions'] = [get_list_of_actions(df)]
    return s

In [ ]:
start_time = time.time()

users_journey = logs_first_fortnight_july[:100000].groupby('idVisit').progress_apply(get_user_journey).droplevel(1)

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# Get list of user without actions and print there logs
users_journey['user_journey_length'] = users_journey['actions'].apply(lambda x: len(x))

users_with_none_actions = users_journey[users_journey['user_journey_length']==0].index.tolist()
print(users_with_none_actions)

for visit in users_with_none_actions[:50]:
    display(logs_first_fortnight_july[logs_first_fortnight_july['idVisit']==visit][[
        'idVisit', 'url', 'type', 'outil', 
    ]])

# Classement des parcours utilisateurs les plus faits

In [ ]:
def get_ranking_users_journey(users_journey):
    users_journey['actions_chained'] = users_journey['actions'].apply(lambda x: ' => '.join(x))
    return users_journey.groupby(by=['entry_point', 'actions_chained']).count().sort_values(['actions'], ascending=False)


In [ ]:
ranking_users_journey = get_ranking_users_journey(users_journey)
ranking_users_journey

# Déterminer puis classer les parcours utilisateurs les plus faits (de bout en bout)

In [ ]:
def get_ranking_users_journey_from_df(df):
    start_time = time.time()

    users_journey = df.groupby('idVisit').progress_apply(get_user_journey).droplevel(1)
    ranking_users_journey = get_ranking_users_journey(users_journey)

    print("--- %s seconds ---" % (time.time() - start_time))
    return ranking_users_journey

In [ ]:
ranking_users_journey = get_ranking_users_journey_from_df(cleaned_logs_july[:500000])
ranking_users_journey[:30]

In [ ]:
ranking_users_journey = get_ranking_users_journey_from_df(cleaned_logs_july)

In [ ]:
ranking_users_journey[:100]

Le faire pour toute l'année semble complexe, on va séparer en trimestes en espérant que ca se passe mieux 

### Premier trimestre : 

In [ ]:
ranking_users_journey_first_quarter = get_ranking_users_journey_from_df(logs_first_quarter)

In [ ]:
ranking_users_journey_first_quarter[:20]

### Second trimestre

In [ ]:
ranking_users_journey_second_quarter = get_ranking_users_journey_from_df(logs_second_quarter)

In [ ]:
ranking_users_journey_second_quarter[:20]

### Troisième trimestre

In [ ]:
ranking_users_journey_third_quarter = get_ranking_users_journey_from_df(logs_third_quarter)

In [ ]:
ranking_users_journey_third_quarter[:20]

### Ensemble de l'année 2022

In [ ]:
ranking_users_journey_2022 = get_ranking_users_journey_from_df(cleaned_logs)

In [ ]:
ranking_users_journey_2022[:30]

In [ ]:
ranking_users_journey_2022.to_excel('test.xlsx', sheet_name='Sheet_name_1')

In [ ]:
# Nombre d'étapes associé au nombre de visiteurs ayant fait autant d'étapes
cleaned_logs.loc[
        (cleaned_logs['idVisit'].shift() != cleaned_logs['idVisit']) |
        (cleaned_logs['url'].shift() != cleaned_logs['url'])
    ].groupby(['idVisit']).agg(
    {'logfile': 'count'}).reset_index().groupby('logfile').count()

# Conclusions

Les résultats semblent etre faux, mais après analyse des parcours utilisateurs les résultats semblent cohérents